# ENVIRONMENT SETUP
*------------------------------------------------------------------------------------*

In [ ]:
# import packages for analysis and modeling
import pandas as pd #data frame operations
import numpy as np #arrays and math functions
import matplotlib.pyplot as plt #2D plotting
%matplotlib inline
import seaborn as sns #
import requests
import os
import io
import string
import re
from datetime import date
from datetime import time
from datetime import datetime

import sys
from os import path

from sklearn.metrics import cohen_kappa_score

In [ ]:
# custome python packages
import rtimbroo_utils_hw5 as rt  

In [ ]:
# set global properties
notebook_file_name = 'awt_sentiment'
report_file_name = 'awt_sentiment'
app_name = 'awt_sentiment'
log_level = 10 # 10-DEBUG, 20-INFO, 30-WARNING, 40-ERROR, 50-CRITICAL

# environment setup
dataDir = './data'
outputDir = './output'
configDir = './config'
logOutDir = './logs'
imageDir = './images'
modelDir = './models'
corpusDir = './corpus'

In [ ]:
# create base output directories if they don't exist
if not os.path.exists(outputDir): os.mkdir(outputDir)
if not os.path.exists(logOutDir): os.mkdir(logOutDir)
if not os.path.exists(imageDir): os.mkdir(imageDir)
if not os.path.exists(modelDir): os.mkdir(modelDir)
if not os.path.exists(dataDir): os.mkdir(dataDir)
if not os.path.exists(configDir): os.mkdir(configDir)
if not os.path.exists(corpusDir): os.mkdir(corpusDir)

In [ ]:
# get current date
now = datetime.utcnow().isoformat()
now_date = re.findall('^[0-9]{4}-[0-9]{2}-[0-9]{2}',now)
now_date

In [ ]:
# get a logger for troubleshooting / data exploration
logger = rt.getFileLogger(logOutDir+'/',app_name+'_'+now_date[0],level=log_level)

## Define Global Functions
*------------------------------------------------------------------------*

In [ ]:
def assign_headline_text_id(text):
    
    if text == "Toyota's LQ Concept Includes Its Own Artificial Intelligence Assistant. This AI will be able to read your emotions to give you the very best mobility experience it can, apparently.":
        text_id = 1
    elif text == "Researchers spy signs of slavery from space. A surge in satellite data and artificial intelligence helps guide enformcement. Here's one important way that space science has improved life on Earth: a surge in satellite data and artifical intelligence has helped put modern slavery under a spotlight.":
        text_id = 2
    elif text == "In a major IVF breakthrough, doctors have started using artificial intelligence to determine the healthiest embryos to transfer to patients!":
        text_id = 3
    elif text == "Bill Gates compares artificial intelligence to nuclear weapons. Gates called AI 'both promissing and dangerous' - and compared the tech to nuclear weapons.":
        text_id = 4
    else:
        text_id = 0
    
    
    return text_id

In [ ]:
def assign_response_text_id(text):
    
    if text == "It's like the design team still thinks it's 1992.":
        text_id = 1
    elif text == "So it disables itself when I want to drive it? Nice":
        text_id = 2
    elif text == "Yeah it's also wearing a bluetooth ear piece":
        text_id = 3
    elif text == 'This is my new response to people that tell me we need to "look after people here on Earth before we explore outer space".':
        text_id = 4
    elif text == "Does it also detect economic slavery from space?":
        text_id = 5
    elif text == "That is not a view from space, science these days uses propaganda":
        text_id = 6
    elif text == "I wonder if my cells would have been good enough to be chosen?":
        text_id = 7
    elif text == "Wow massive I was lucky to have 2 implanted after many years of ivf and the took resulting in 2 babies who drive me crazy ?? but I love them more every day":
        text_id = 8
    elif text == "How unethical!":
        text_id = 9
    elif text == "Of course it will bite us in the ass eventually, but until then it should be very exciting.":
        text_id = 10
    elif text == "pure nonsense! ..it's no different than guns, dependent on how you legislate, it will be safe or end in mass shootings\nswiss has the safest gun control!!":
        text_id = 11
    elif text == "Doesn't sound like something we can ethically handle yet.":
        text_id = 12
    else:
        text_id = 0
    
    
    return text_id

In [ ]:
def is_nan(x):
    return (x is np.nan or x != x)

In [ ]:
def assign_label_value(label):
    
    if is_nan(label):
        label_value = 0
    elif label == 'Negative':
        label_value = 1
    elif label == 'Neutral':
        label_value = 2
    elif label == 'Positive':
        label_value = 3
    
    return label_value

# OBTAIN the DATA
*-----------------------------------------------------------------------------*

In [ ]:
# AMT Turk Data
artical_headline_labels = pd.read_csv(f'{dataDir}/ArticalHeadlines_Batch_3824170_batch_results.csv')
response_post_labels = pd.read_csv(f'{dataDir}/ResponsePosts_Batch_3822162_batch_results.csv')
workers = pd.read_csv(f'{dataDir}/User_1466364_workers.csv')



In [ ]:
logger.info(f'artical_headline_labels shape: {artical_headline_labels.shape}')
logger.info(f'artical_headline_labels size: {artical_headline_labels.size}')
artical_headline_labels.head()

In [ ]:
keep_cols_art = ['Answer.sentiment.label','Input.text','WorkerId','WorkTimeInSeconds','LifetimeApprovalRate']
artical_headline_df = artical_headline_labels[keep_cols_art]
#logger.debug(artical_headline_df['Input.text'].unique())
artical_headline_df['text_id'] = [assign_headline_text_id(t) for t in artical_headline_df['Input.text'] ]
artical_headline_df['label_value'] = [assign_label_value(l) for l in artical_headline_df['Answer.sentiment.label'] ]
artical_headline_df.head()

In [ ]:
#len(artical_headline_df['WorkerId'].unique())
workers = artical_headline_df['WorkerId'].unique()
logger.info(f'unique workers: {workers}')
logger.info(f'Number of unique workers: {len(workers)}')
worker_labels_art =  artical_headline_df[['WorkerId','text_id','label_value']]
worker_labels_art = worker_labels_art.sort_values(by=['WorkerId'])
#logger.debug(worker_labels)
#worker_labels

In [ ]:
sns.barplot(x='text_id', y='label_value' , data=worker_labels_art)

In [ ]:
workers_comp = []
worker_labels_art_keep = pd.DataFrame()

for worker in workers:
    #print(worker)
    _w = worker_labels_art[worker_labels_art.WorkerId == worker]
    if len(_w) == 4:
        workers_comp.append(_w)
        worker_labels_art_keep = pd.concat([worker_labels_art_keep,_w])
        #print()
    
logger.info(f'{len(workers_comp)} full sets')
logger.info(f'worker_labels_art_keep shape: {worker_labels_art_keep.shape}')
worker_labels_art_keep = worker_labels_art_keep.sort_values(by=['WorkerId','text_id'])
worker_labels_art_keep.head(10)

In [ ]:
sns.barplot(x='text_id', y='label_value' , data=worker_labels_art_keep);

In [ ]:
t = worker_labels_art_keep.groupby(by='WorkerId')['label_value'].mean()
t

In [ ]:
def calc_kappa(w1,w2):
    w1_label_values = []
    w2_label_values = []
    
    for w1_k,w1_v in w1.items():
        w1_label_values.append(w1_v)
    
    for w2_k,w2_v in w2.items():
        w2_label_values.append(w2_v)
    
    
    logger.info(f'\nkappa pair-wise comparision:\n {w1_label_values} \n {w2_label_values}')
    k_score = cohen_kappa_score(w1_label_values,w2_label_values)
    logger.info(f'kappa score: {k_score}\n')
    return k_score

In [ ]:
#(w1,w2,score)
workers = worker_labels_art_keep['WorkerId'].unique()
coder_pairs = []
workers_dict = {}

for worker in workers:
    w = worker_labels_art_keep[worker_labels_art_keep.WorkerId == worker]
    workers_dict[worker] = dict(zip(w['text_id'],w['label_value']))
    
logger.info(f'Worker Text Labels: {workers_dict}')

In [ ]:
k_score_pairs = []
num_workers = len(workers)

for i,w_id in enumerate(workers):
    logger.info(f' index: {i}')
    logger.info(f' worker id: {w_id}')
    this_wId = w_id
    logger.info(f' worker labels: {workers_dict[w_id]}')
    
    # get next worker to compare to
    for i_n in range(i+1, num_workers):
        
        logger.info(f' next worker id: {workers[i_n]}')
        next_wId = workers[i_n]
        logger.info(f' next worker labels: {workers_dict[next_wId]}')

        k_score_pairs.append((this_wId,next_wId,calc_kappa(workers_dict[w_id],workers_dict[next_wId])))
    
    #if i == 2: break
k_score_pairs    
logger.info(f'k_score_pairs count: {len(k_score_pairs)}')
   

In [ ]:
logger.info(k_score_pairs)

In [ ]:
# get average of kappa scores
scores = []
for score in k_score_pairs:
    scores.append(score[2])
    
logger.info(f'Kappa PAIR-WISE AVG SCORE:: Sentiment Analysis on Social Media Headline Posts: {np.array(scores).mean()}')
scores_sorted = np.sort(np.array(scores))
logger.info(f'Kappa PAIR-WISE MEDIAN SCORE: Sentiment Analysis on Social Media Headline Posts: {np.median(scores_sorted)}')

In [ ]:
logger.info(f'response_post_labels shape: {response_post_labels.shape}')
logger.info(f'response_post_labels size: {response_post_labels.size}')
response_post_labels.head()

In [ ]:
keep_cols_resp = ['Answer.sentiment.label','Input.article_headline','Input.text','WorkerId','WorkTimeInSeconds','LifetimeApprovalRate']
response_post_df = response_post_labels[keep_cols_resp]
response_post_df['article_text_id'] = [assign_headline_text_id(t) for t in response_post_df['Input.article_headline'] ]
response_post_df['response_text_id'] = [assign_response_text_id(t) for t in response_post_df['Input.text'] ]
response_post_df['label_value'] = [assign_label_value(l) for l in response_post_df['Answer.sentiment.label'] ]
response_post_df.head()

In [ ]:
#len(artical_headline_df['WorkerId'].unique())
workers_resp = response_post_df['WorkerId'].unique()
logger.info(f'Unique Response Workers: {len(workers_resp)}')
worker_labels_resp =  response_post_df[['WorkerId','response_text_id','label_value','article_text_id']]
worker_labels_resp = worker_labels_resp.sort_values(by=['WorkerId'])
logger.info(f'worker_labels_resp shape: {worker_labels_resp.shape}')
#logger.debug(worker_labels_resp)
#worker_labels_resp

In [ ]:
sns.barplot(x='response_text_id', y='label_value' , data=worker_labels_resp)

In [ ]:
logger.debug(response_post_df['Input.text'].unique())
response_post_df['Answer.sentiment.label'].unique()

In [ ]:
workers_resp_full = []
worker_labels_resp_keep = pd.DataFrame()

for worker in workers_resp:
    #print(worker)
    _w = worker_labels_resp[worker_labels_resp.WorkerId == worker]
    if len(_w) == 12:
        workers_resp_full.append(_w)
        worker_labels_resp_keep = pd.concat([worker_labels_resp_keep,_w])
        #print()
    
logger.info(f'{len(workers_resp_full)} full sets')
logger.info(f'worker_labels_resp_keep shape: {worker_labels_resp_keep.shape}')
worker_labels_resp_keep = worker_labels_resp_keep.sort_values(by=['WorkerId','response_text_id'])
worker_labels_resp_keep.head(10)

In [ ]:
# plot of kept labeled dataset
sns.barplot(x='response_text_id', y='label_value' , data=worker_labels_resp_keep);

In [ ]:
# calculate kappa
#(w1,w2,score)
workers = worker_labels_resp_keep['WorkerId'].unique()
coder_pairs = []
workers_dict = {}

for worker in workers:
    w = worker_labels_resp_keep[worker_labels_resp_keep.WorkerId == worker]
    workers_dict[worker] = dict(zip(w['response_text_id'],w['label_value']))
    
logger.info(f'Worker Text Labels: {workers_dict}')

In [ ]:
k_score_pairs = []
num_workers = len(workers)

for i,w_id in enumerate(workers):
    logger.info(f' index: {i}')
    logger.info(f' worker id: {w_id}')
    this_wId = w_id
    logger.info(f' worker labels: {workers_dict[w_id]}')
    
    # get next worker to compare to
    for i_n in range(i+1, num_workers):
        
        logger.info(f' next worker id: {workers[i_n]}')
        next_wId = workers[i_n]
        logger.info(f' next worker labels: {workers_dict[next_wId]}')

        k_score_pairs.append((this_wId,next_wId,calc_kappa(workers_dict[w_id],workers_dict[next_wId])))
    
    #if i == 2: break

k_score_pairs    
logger.info(f'number of pairs calculated: {len(k_score_pairs)}')
   

In [ ]:
# get average of kappa scores
scores = []
for score in k_score_pairs:
    scores.append(score[2])
    
logger.info(f'Kappa PAIR-WISE AVG SCORE: Public Opinion Toward AI: {np.array(scores).mean()}')
scores_sorted = np.sort(np.array(scores))
logger.info(f'Kappa PAIR-WISE MEDIAN SCORE: Public Opinion Toward AI: {np.median(scores_sorted)}')
